### Melon Top100 chart
- 100곡의 노래 제목과 ID 추출 후, 저장
- 100곡의 노래의 상세 정보를 추출해서 list와 dict에 저장 후, json 파일로 저장
- json 파일을 load하여 Pandas의 DataFrame에 저장
- DataFrame 객체를 DB의 table에 저장
#### 정규 표현식
- https://www.nextree.co.kr/p4327/

In [2]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더(브라우저 정보)
   'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
# print(res.status_code)

if(res.ok):
    html = res.text # url의 elments 모든 text 가져오기
    # BeautifulSoup 생성자에서 html을 파싱하고 그 결과를 BeautifulSoup 객체로 반환한다.
    soup = BeautifulSoup(html, 'html.parser')
    # playSong이 들어간 a href태그(div태그의 id가 tb_list인 tr안에 있음)
    print(len(soup.select("div#tb_list tr a[href*='playSong']")))
    a_tags = soup.select("div#tb_list tr a[href*='playSong']")
    
    song_list=[]
    for idx, a_tag in enumerate(a_tags,1):
        song_title = a_tag.text # a 태그안에 text요소(노래 제목) 가져오기
        song_dict={}
        song_dict['title'] = song_title
        href_value = a_tag['href']  # a 태그안에 href요소 가져오기
        # print(href_value)
        # 정규표현식 사용(re)해서 노래 ID찾기
        matched = re.search(r'(\d+)\);', href_value)
        # print(matched.group(1))
        if matched:
            song_id = matched.group(1)
            song_dict['id'] = song_id
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['url'] = song_detail_url
            # print(idx, song_title, song_id, song_detail_url)
            print(song_dict)
            song_list.append(song_dict)
print(len(song_list))

100
{'title': '사랑은 늘 도망가', 'id': '34061322', 'url': 'https://www.melon.com/song/detail.htm?songId=34061322'}
{'title': '취중고백', 'id': '34431086', 'url': 'https://www.melon.com/song/detail.htm?songId=34431086'}
{'title': '호랑수월가', 'id': '34535898', 'url': 'https://www.melon.com/song/detail.htm?songId=34535898'}
{'title': 'Step Back', 'id': '34494149', 'url': 'https://www.melon.com/song/detail.htm?songId=34494149'}
{'title': '회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)', 'id': '34256568', 'url': 'https://www.melon.com/song/detail.htm?songId=34256568'}
{'title': 'ELEVEN', 'id': '34349913', 'url': 'https://www.melon.com/song/detail.htm?songId=34349913'}
{'title': '리무진 (Feat. MINO) (Prod. GRAY)', 'id': '34298499', 'url': 'https://www.melon.com/song/detail.htm?songId=34298499'}
{'title': 'Counting Stars (Feat. Beenzino)', 'id': '34398735', 'url': 'https://www.melon.com/song/detail.htm?songId=34398735'}
{'title': 'Dreams Come True', 'id': '34436674', 'url': 'https://www.melon.com/song/detail.htm?son

In [18]:
import requests
from bs4 import BeautifulSoup
import re

req_header_dict = {
    # 요청헤더(브라우저 정보)
   'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list[:1],1):
    song_detail_dict = {} # 노래 1곡의 상세정보를 저장하는 dict
    song_detail_url = song['url']
    
    res = requests.get(song_detail_url, headers=req_header_dict)    
    print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        song_detail_dict['곡명'] = song['title']
        singer_span = soup.select("a[href*='goArtistDetail'] span")
        if singer_span:
            song_detail_dict['가수'] = singer_span[0].text
        #print(song_detail_dict)
        
        #print(soup.select("div.meta dd"))
        song_dd = soup.select("div.meta dd")
        if song_dd:
            # print(song_dd[0].text)
            song_detail_dict['앨범'] = song_dd[0].text
            song_detail_dict['발매일'] = song_dd[1].text
            song_detail_dict['장르'] = song_dd[2].text
        #print(song_detail_dict)
        song_detail_list.append(song_detail_dict)        
        
        song_id = song['id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        if like_res.ok:
            #print(like_res.json())
            song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
        print(song_detail_dict)
        song_detail_list.append(song_detail_dict)

200
{'곡명': '사랑은 늘 도망가', '가수': '임영웅', '앨범': '신사와 아가씨 OST Part.2', '발매일': '2021.10.11', '장르': '발라드, 국내드라마', '좋아요': 123935}
